In [2]:
!pip install haversine

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [301]:
from haversine import haversine ## 좌표로 거리 구하기 300m

In [302]:
haversine((127.0016985,37.5642135), (127.0016985,37.5686963), unit = 'km') 

0.29999571050047735

In [303]:
## 500미터에 해당하는 좌표 차이
xx = 0.0044966
yy = 0.0074714

In [541]:
## 100미터에 해당하는 좌표 차이
xxx = xx/5
yyy = yy/4

In [540]:
# 좌표를 중심으로한 1km^2 사각형의 꼭지점 4개의 좌표 구하기

def square(x,y,xx= 0.0044966,yy=0.0074714):
  s_list = []
  s_list.append([x-xx,y])
  s_list.append([x+xx,y])
  s_list.append([x,y-yy])
  s_list.append([x,y+yy])
  return s_list

In [305]:
square(127.0016985,37.5642135)

[[126.99720190000001, 37.5642135],
 [127.0061951, 37.5642135],
 [127.0016985, 37.5567421],
 [127.0016985, 37.5716849]]

In [306]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/elice_miniProject/metro_station_seoul.csv', encoding="cp949")

In [307]:
df

,Unnamed: 0,metro_station_name,metro_station_addr,metro_geo_lat,metro_geo_lng
0,0,강남역 2호선,서울 강남구 강남대로 지하 396,37.498165,127.028308
1,1,서울대입구역 2호선,서울 관악구 남부순환로 지하 1822,37.481153,126.952645
2,2,합정역 2호선,서울 마포구 양화로 지하 55,37.550115,126.914638
3,3,왕십리역 2호선,서울 성동구 왕십리로 지하 300,37.561222,127.036655
4,4,당산역 2호선,서울 영등포구 당산로 229,37.534957,126.902731
...,...,...,...,...,...
439,439,신당역 6호선,서울 중구 다산로 지하 260,37.566274,127.016154
440,440,상봉역 경춘선,서울 중랑구 망우로 297,37.596714,127.085437
441,441,강남구청역 수인분당선,서울 강남구 학동로 지하 346,37.517189,127.041301
442,442,신설동역 2호선,서울 동대문구 왕산로 지하 1,37.575319,127.024702


In [308]:
del df['Unnamed: 0'] ## 필요 없는 열 삭제

In [309]:
df['station_num'] = '' ## 호선 번호 열 생성

In [310]:
## 역 이름에서 역이름, 호선 번호로 열 분리
de = []
for i in range(len(df['metro_station_name'])):
  try:
    df['station_num'][i]=df['metro_station_name'][i].split()[1]
    df['metro_station_name'][i]=df['metro_station_name'][i].split()[0]
  except:
    de.append(i) ## 호선 정보가 없는 행 번호

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [311]:
print(de)

[254, 320, 354, 392, 426]


In [312]:
de.sort(reverse=True)
de

[426, 392, 354, 320, 254]

In [313]:
## 호선 정보가 없는 열 삭제 후 인덱스 번호 초기화

for i in range(len(de)):
  df.drop(df.index[i],inplace=True)
df.reset_index(drop=True, inplace=True)

In [314]:
df

,metro_station_name,metro_station_addr,metro_geo_lat,metro_geo_lng,station_num
0,서울대입구역,서울 관악구 남부순환로 지하 1822,37.481153,126.952645,2호선
1,왕십리역,서울 성동구 왕십리로 지하 300,37.561222,127.036655,2호선
2,신촌역,서울 마포구 신촌로 지하 90,37.555442,126.936119,2호선
3,신림역,서울 관악구 남부순환로 지하 1614,37.484218,126.929803,2호선
4,교대역,서울 서초구 서초대로 지하 294,37.493577,127.014156,3호선
...,...,...,...,...,...
434,신당역,서울 중구 다산로 지하 260,37.566274,127.016154,6호선
435,상봉역,서울 중랑구 망우로 297,37.596714,127.085437,경춘선
436,강남구청역,서울 강남구 학동로 지하 346,37.517189,127.041301,수인분당선
437,신설동역,서울 동대문구 왕산로 지하 1,37.575319,127.024702,2호선


In [319]:
## 호선별 지하철역을 빠르게 구하기 위해 딕셔너리로
## idx로 호선별 index번호 추가 

metro = {}
for i in range(len(df['metro_station_name'])):
  if df['station_num'][i] in metro:
    idx = len(metro[df['station_num'][i]])
    a = {idx:[df['metro_station_name'][i],(df['metro_geo_lng'][i],df['metro_geo_lat'][i])]}
    metro[df['station_num'][i]].append(a)
    
  else:
    idx = 0 
    a = {idx:[df['metro_station_name'][i],(df['metro_geo_lng'][i],df['metro_geo_lat'][i])]}
    metro[df['station_num'][i]]=[a]
  

In [320]:
metro['2호선'][0] 

{0: ['서울대입구역', (126.952645104944, 37.481152663184396)]}

In [321]:
metro['2호선'][0][0]

['서울대입구역', (126.952645104944, 37.481152663184396)]

In [322]:
metro['2호선'][0][0][1][0], metro['2호선'][0][0][1][1] ## 서울대 입구역의 x, y 좌표

(126.952645104944, 37.481152663184396)

In [323]:
a = square(metro['2호선'][0][0][1][0],metro['2호선'][0][0][1][1])
a

[[126.948148504944, 37.481152663184396],
 [126.957141704944, 37.481152663184396],
 [126.952645104944, 37.473681263184396],
 [126.952645104944, 37.488624063184396]]

In [324]:
metro['2호선'][0][0][1][0],metro['2호선'][0][0][1][1]

(126.952645104944, 37.481152663184396)

In [325]:
df.columns = ['name', 'addr','lat','lng','station_num']
df

,name,addr,lat,lng,station_num
0,서울대입구역,서울 관악구 남부순환로 지하 1822,37.481153,126.952645,2호선
1,왕십리역,서울 성동구 왕십리로 지하 300,37.561222,127.036655,2호선
2,신촌역,서울 마포구 신촌로 지하 90,37.555442,126.936119,2호선
3,신림역,서울 관악구 남부순환로 지하 1614,37.484218,126.929803,2호선
4,교대역,서울 서초구 서초대로 지하 294,37.493577,127.014156,3호선
...,...,...,...,...,...
434,신당역,서울 중구 다산로 지하 260,37.566274,127.016154,6호선
435,상봉역,서울 중랑구 망우로 297,37.596714,127.085437,경춘선
436,강남구청역,서울 강남구 학동로 지하 346,37.517189,127.041301,수인분당선
437,신설동역,서울 동대문구 왕산로 지하 1,37.575319,127.024702,2호선


In [326]:
## 좌표 반경 500미터 내의 지하철 역, 100미터 내의 버스 정류장 찾기

def round500(x,y,df,bus=False): ## x,y좌표 , DataFrame변수명, True == 버스, False == 지하철

  box = square(x,y) # 위치 좌표를 중심으로한 1km^2 정사각형의 꼭지점 4개의 좌표 구하기

  station = []
  left_x = box[0][0] # 좌측 x좌표
  right_x = box[1][0] # 우측 x좌표
  low_y = box[2][1] # 하단 y좌표
  hight_y = box[3][1] # 상단 y좌표
  result = []

  df = df.sort_values(by=["lng", "lat"], ascending=[True, True])  ## x좌표 기준으로 데이터 프레임 sorting
  df.reset_index(drop=True, inplace=True)

  for i in range(len(df['name'])): 
    if df['lng'][i] >=  right_x: ## x가 범위 500m 좌표 내에 없을 시 break
      break

    if df['lng'][i] >= left_x and df['lng'][i] < right_x  and df['lat'][i] >= low_y and df['lat'][i] < hight_y:
      station.append(i) ## 구해놓은 꼭지점 내에 해당하는 역의 index번호를 station 리스트에 저장
  
  for i in station: ## 1km^2 범위 내에서 선택한 위치를 중심으로 반경 내의 역만

    if bus==True: ## 버스면 100미터 지하철은 500미터
      distance = 0.1
    else:
      distance = 0.5
    if abs(haversine((df['lng'][i],df['lat'][i]), (x, y), unit = 'km')) <= distance:
      result.append(i)
  return result

In [453]:
round500(metro['2호선'][0][0][1][0],metro['2호선'][0][0][1][1],df) ## 서울대입구 역 반경 500미터 내의 지하철 역 찾기

[123]

In [454]:
df['name'][123]

'서울대입구역'

In [329]:
df = df.sort_values(by=["lng", "lat"], ascending=[True, True])  ## 함수와 같이 sorting
df.reset_index(drop=True, inplace=True)
df['station_num'][123] ## 서울대입구 역 반경 500m에는 사당역 말고 없는듯

'2호선'

In [330]:
print(metro[df['station_num'][161]]) ## metro에서 2호선의 모든 역 보기

[{0: ['서울대입구역', (126.952645104944, 37.481152663184396)]}, {1: ['왕십리역', (127.036655378283, 37.561221741519105)]}, {2: ['신촌역', (126.936119192554, 37.555441786598394)]}, {3: ['신림역', (126.92980331103, 37.4842176141092)]}, {4: ['삼성역', (127.063073009788, 37.5088619974404)]}, {5: ['역삼역', (127.036618203089, 37.500716269359295)]}, {6: ['성수역', (127.05609047122299, 37.5445023983984)]}, {7: ['사당역', (126.981408909449, 37.476522834761894)]}, {8: ['강변역', (127.094664569096, 37.5351692767888)]}, {9: ['대림역', (126.8949461878, 37.4933105886136)]}, {10: ['건대입구역', (127.06917328155399, 37.540418887217605)]}, {11: ['구로디지털단지역', (126.901494378412, 37.485314923786504)]}, {12: ['선릉역', (127.048942471228, 37.5045028775835)]}, {13: ['신대방역', (126.913175585996, 37.4875759745635)]}, {14: ['방배역', (126.99769489932699, 37.4814364876589)]}, {15: ['잠실새내역', (127.08637386720699, 37.5115672093323)]}, {16: ['서초역', (127.007580814715, 37.491822999009294)]}, {17: ['동대문역사문화공원역', (127.009131498424, 37.565589522977604)]}, {18: ['낙성대역

In [331]:
 ##해당 역의 호선 내 모든 역을 중심으로하는 1km^2 정사각형의 꼭지점 

def all_station(df_index):
  all = []
  for i in range(len(metro[df['station_num'][df_index]])):
    all.append(square(metro[df['station_num'][df_index]][i][i][1][0],metro[df['station_num'][df_index]][i][i][1][1]))
  return all

In [332]:
print(all_station(161)) # 2호선 모든 역의 정사각형 좌표

[[[126.948148504944, 37.481152663184396], [126.957141704944, 37.481152663184396], [126.952645104944, 37.473681263184396], [126.952645104944, 37.488624063184396]], [[127.032158778283, 37.561221741519105], [127.041151978283, 37.561221741519105], [127.036655378283, 37.553750341519105], [127.036655378283, 37.568693141519105]], [[126.931622592554, 37.555441786598394], [126.940615792554, 37.555441786598394], [126.936119192554, 37.547970386598394], [126.936119192554, 37.562913186598394]], [[126.92530671103, 37.4842176141092], [126.93429991103, 37.4842176141092], [126.92980331103, 37.4767462141092], [126.92980331103, 37.4916890141092]], [[127.058576409788, 37.5088619974404], [127.067569609788, 37.5088619974404], [127.063073009788, 37.5013905974404], [127.063073009788, 37.5163333974404]], [[127.03212160308901, 37.500716269359295], [127.041114803089, 37.500716269359295], [127.036618203089, 37.493244869359295], [127.036618203089, 37.508187669359295]], [[127.05159387122299, 37.5445023983984], [127

In [333]:
print(metro[df['station_num'][161]])

[{0: ['서울대입구역', (126.952645104944, 37.481152663184396)]}, {1: ['왕십리역', (127.036655378283, 37.561221741519105)]}, {2: ['신촌역', (126.936119192554, 37.555441786598394)]}, {3: ['신림역', (126.92980331103, 37.4842176141092)]}, {4: ['삼성역', (127.063073009788, 37.5088619974404)]}, {5: ['역삼역', (127.036618203089, 37.500716269359295)]}, {6: ['성수역', (127.05609047122299, 37.5445023983984)]}, {7: ['사당역', (126.981408909449, 37.476522834761894)]}, {8: ['강변역', (127.094664569096, 37.5351692767888)]}, {9: ['대림역', (126.8949461878, 37.4933105886136)]}, {10: ['건대입구역', (127.06917328155399, 37.540418887217605)]}, {11: ['구로디지털단지역', (126.901494378412, 37.485314923786504)]}, {12: ['선릉역', (127.048942471228, 37.5045028775835)]}, {13: ['신대방역', (126.913175585996, 37.4875759745635)]}, {14: ['방배역', (126.99769489932699, 37.4814364876589)]}, {15: ['잠실새내역', (127.08637386720699, 37.5115672093323)]}, {16: ['서초역', (127.007580814715, 37.491822999009294)]}, {17: ['동대문역사문화공원역', (127.009131498424, 37.565589522977604)]}, {18: ['낙성대역

In [334]:
df2 = pd.read_csv('/content/drive/My Drive/elice_miniProject/bus_stop_seoul.csv', encoding="cp949")

In [335]:
df2

,Unnamed: 0,bus_stop_name,bus_stop_standard_id,bus_stop_ars_id,bus_stop_geo_lat,bus_stop_geo_lng
0,0,종로2가사거리,100000001,1001,37.569765,126.987750
1,1,창경궁.서울대학교병원,100000002,1002,37.579183,126.996566
2,2,명륜3가.성대입구,100000003,1003,37.582671,126.998340
3,3,종로2가.삼일교,100000004,1004,37.568579,126.987613
4,4,혜화동로터리.여운형활동터,100000005,1005,37.586243,127.001744
...,...,...,...,...,...,...
11174,11174,고덕1동주민센터.두레근린공원,124900120,25529,37.558546,127.151715
11175,11175,고덕역1번출구,124900121,25518,37.555506,127.153072
11176,11176,고덕역1번출구,124900122,25528,37.555506,127.153378
11177,11177,명일신동아아파트,124900123,25520,37.552350,127.152546


In [336]:
del df2['Unnamed: 0']

In [337]:
len(round500(metro['2호선'][0][0][1][0],metro['2호선'][0][0][1][1],df))

1

In [338]:
box = square(metro['2호선'][0][0][1][0],metro['2호선'][0][0][1][1])
box

[[126.948148504944, 37.481152663184396],
 [126.957141704944, 37.481152663184396],
 [126.952645104944, 37.473681263184396],
 [126.952645104944, 37.488624063184396]]

In [339]:
df2.columns = ['name', 'standard_id','ars_id','lat','lng']
df2

,name,standard_id,ars_id,lat,lng
0,종로2가사거리,100000001,1001,37.569765,126.987750
1,창경궁.서울대학교병원,100000002,1002,37.579183,126.996566
2,명륜3가.성대입구,100000003,1003,37.582671,126.998340
3,종로2가.삼일교,100000004,1004,37.568579,126.987613
4,혜화동로터리.여운형활동터,100000005,1005,37.586243,127.001744
...,...,...,...,...,...
11174,고덕1동주민센터.두레근린공원,124900120,25529,37.558546,127.151715
11175,고덕역1번출구,124900121,25518,37.555506,127.153072
11176,고덕역1번출구,124900122,25528,37.555506,127.153378
11177,명일신동아아파트,124900123,25520,37.552350,127.152546


In [452]:
round500(metro['2호선'][0][0][1][0],metro['2호선'][0][0][1][1],df2,True)

[4371, 4373, 4379]

In [342]:
len(round500(127.952645104944, 38.481152663184396,df))

0

In [413]:
## 버스정류장

def all_bus_stop(df2,df_index): 
  return square(df2['lng'][df_index],df2['lat'][df_index])

In [344]:
all_bus_stop(4371)

[[126.9144923746, 37.6367680033],
 [126.9234855746, 37.6367680033],
 [126.9189889746, 37.6292966033],
 [126.9189889746, 37.6442394033]]

In [345]:
# 500미터 내의 치하철, 100미터 내의 버스정류장 모두 찾기

def result(x,y,df_metro,df_bus):
  result = []

  metro = round500(x,y,df_metro) ## 반경 100미터 지하철역 찾기
  for station in metro:
    result = result + all_station(station)

  bus = round500(x,y,df_bus,True) ##반경 100미터 버스정류장 찾기
  for stop in bus:
    result += all_bus_stop(stop)
  
  return result


In [346]:
print(result(metro['2호선'][0][0][1][0],metro['2호선'][0][0][1][1],df,df2))

[[[126.948148504944, 37.481152663184396], [126.957141704944, 37.481152663184396], [126.952645104944, 37.473681263184396], [126.952645104944, 37.488624063184396]], [[127.032158778283, 37.561221741519105], [127.041151978283, 37.561221741519105], [127.036655378283, 37.553750341519105], [127.036655378283, 37.568693141519105]], [[126.931622592554, 37.555441786598394], [126.940615792554, 37.555441786598394], [126.936119192554, 37.547970386598394], [126.936119192554, 37.562913186598394]], [[126.92530671103, 37.4842176141092], [126.93429991103, 37.4842176141092], [126.92980331103, 37.4767462141092], [126.92980331103, 37.4916890141092]], [[127.058576409788, 37.5088619974404], [127.067569609788, 37.5088619974404], [127.063073009788, 37.5013905974404], [127.063073009788, 37.5163333974404]], [[127.03212160308901, 37.500716269359295], [127.041114803089, 37.500716269359295], [127.036618203089, 37.493244869359295], [127.036618203089, 37.508187669359295]], [[127.05159387122299, 37.5445023983984], [127

In [387]:
df3 = pd.read_csv('/content/drive/My Drive/elice_miniProject/bus_line.csv', encoding="cp949")

In [393]:
df3.columns = ['standard_id','line_name','num','section_id','bus_stop_id','ars_id','name_y','lat_y','lng_y']

In [394]:
df3

,standard_id,line_name,num,section_id,bus_stop_id,ars_id,name_y,lat_y,lng_y
0,100100124,0017,1,0,102000271,3689,청암자이아파트,126.946517,37.534363
1,100100124,0017,2,102700549,102000204,3298,청암동강변삼성아파트,126.949304,37.533961
2,100100124,0017,3,102700550,102000227,3321,청심경로당,126.950449,37.533744
3,100100124,0017,4,102700551,102000210,3304,원효2동주민센터,126.950904,37.534278
4,100100124,0017,5,102700552,102000212,3306,산천동,126.953984,37.535420
...,...,...,...,...,...,...,...,...,...
39358,106900002,중랑02,14,106700969,106900003,7525,면목지구대,127.094202,37.587838
39359,106900002,중랑02,15,106700970,106900025,7526,새마을금고,127.097335,37.589118
39360,106900002,중랑02,16,106700044,106900002,7527,공판장,127.099060,37.588522
39361,106900002,중랑02,17,106700045,106900026,7528,진주빌라.바다약국,127.100669,37.587783


In [395]:
df2

,name,standard_id,ars_id,lat,lng
0,종로2가사거리,100000001,1001,37.569765,126.987750
1,창경궁.서울대학교병원,100000002,1002,37.579183,126.996566
2,명륜3가.성대입구,100000003,1003,37.582671,126.998340
3,종로2가.삼일교,100000004,1004,37.568579,126.987613
4,혜화동로터리.여운형활동터,100000005,1005,37.586243,127.001744
...,...,...,...,...,...
11174,고덕1동주민센터.두레근린공원,124900120,25529,37.558546,127.151715
11175,고덕역1번출구,124900121,25518,37.555506,127.153072
11176,고덕역1번출구,124900122,25528,37.555506,127.153378
11177,명일신동아아파트,124900123,25520,37.552350,127.152546


In [396]:
merge = pd.merge(df2, df3,
 how='inner',
 on='ars_id')

In [397]:
merge

,name,standard_id_x,ars_id,lat,lng,standard_id_y,line_name,num,section_id,bus_stop_id,name_y,lat_y,lng_y
0,종로2가사거리,100000001,1001,37.569765,126.987750,100100073,470,65,101701943,100000001,종로2가사거리,126.987786,37.569764
1,종로2가사거리,100000001,1001,37.569765,126.987750,123000010,741,76,101701943,100000001,종로2가사거리,126.987786,37.569764
2,종로2가사거리,100000001,1001,37.569765,126.987750,100100585,N37,33,101701943,100000001,종로2가사거리,126.987786,37.569764
3,창경궁.서울대학교병원,100000002,1002,37.579183,126.996566,100100549,100,31,100700020,100000002,창경궁.서울대학교병원,126.996520,37.579179
4,창경궁.서울대학교병원,100000002,1002,37.579183,126.996566,100100007,102,31,100700020,100000002,창경궁.서울대학교병원,126.996520,37.579179
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34127,고덕2단지,124900110,25988,37.560222,127.162899,124900003,강동02,57,124700074,124900110,고덕2단지,127.162906,37.560282
34128,고덕2단지,124900110,25988,37.560222,127.162899,124900001,강동05,53,124700074,124900110,고덕2단지,127.162906,37.560282
34129,가래여울,124900111,25760,37.576564,127.169013,124900003,강동02,68,124701425,124900111,가래여울,127.168974,37.576545
34130,고덕래미안힐스테이트.시니어센터,124900112,25732,37.559048,127.147861,124900003,강동02,52,124701403,124900112,고덕래미안힐스테이트.시니어센터,127.148073,37.559227


In [491]:
## 버스 번호로 검색이 가능한 딕셔너리
bus = {}
for i in range(len(merge['ars_id'])):

  if merge['line_name'][i] in bus:
    idx = len(bus[merge['line_name'][i]])
    a = [merge['ars_id'][i],(merge['lng'][i],merge['lat'][i])]
    bus[merge['line_name'][i]].append(a)
    
  else:
    idx = 0 
    a = [ merge['ars_id'][i],(merge['lng'][i],merge['lat'][i])]
    bus[merge['line_name'][i]]=[a]
  

In [484]:
## 버스 정류장 번호로 검색 가능한 딕셔너리

bus2 = {}
for i in range(len(merge['ars_id'])):

  if merge['ars_id'][i] in bus2:
    idx = len(bus2[merge['ars_id'][i]])
    a = [merge['line_name'][i],(merge['lng'][i],merge['lat'][i])]
    bus2[merge['ars_id'][i]].append(a)
    
  else:
    idx = 0 
    a = [merge['line_name'][i],(merge['lng'][i],merge['lat'][i])]
    bus2[merge['ars_id'][i]]=[a]
  

In [550]:
round500(metro['2호선'][0][0][1][0],metro['2호선'][0][0][1][1],merge,True) ## 서울대입구역 근처 100미터 내 버스정류장의 인덱스

[13611, 13630, 13631, 13632, 13665, 13666, 13667, 13668, 13669]

In [493]:
metro['2호선'][0]

{0: ['서울대입구역', (126.952645104944, 37.481152663184396)]}

In [551]:
merge['ars_id'][13611] ## 서울대입구역 근처 인덱스13611번의 버스정류장 번호

12254

In [556]:
bus2[merge['ars_id'][13611]][0][0]


'7022'

In [495]:
bus2[merge['ars_id'][13611]][0][0] ## ars_id 12254 버스정류장의 인덱스 0번 버스 번호

'7022'

In [557]:
print(bus[bus2[merge['ars_id'][13611]][0][0]]) ## 7022번 버스가 다니는 버스 노선의 버스 정류장 번호 

[[1102, (126.97026536870001, 37.5832657654)], [1104, (126.9710436571, 37.585113193000005)], [1105, (126.9725135095, 37.5878519552)], [1106, (126.96712618049999, 37.5921866176)], [1107, (126.9671906625, 37.5923353152)], [1108, (126.97246131979999, 37.5881285775)], [1109, (126.9711546793, 37.584966342600005)], [1110, (126.97105004040002, 37.5803257033)], [1111, (126.9713851083, 37.580513211799996)], [1112, (126.9706209712, 37.583091043)], [1115, (126.97219601, 37.5768666917)], [1116, (126.9722828893, 37.577671386300004)], [1126, (126.9765652658, 37.5726577416)], [1130, (126.9820170203, 37.5759017489)], [1132, (126.9608880061, 37.6046629766)], [1133, (126.9612198463, 37.602502279899994)], [1136, (126.96170682350001, 37.598133066500004)], [1137, (126.9636434993, 37.5955935123)], [1138, (126.9639574356, 37.5927380599)], [1139, (126.96514192110001, 37.5927861733)], [1140, (126.9639775924, 37.5955575712)], [1141, (126.962215308, 37.598147647800005)], [1142, (126.9592405649, 37.5996163318)], [

In [509]:
bus[bus2[merge['ars_id'][13611]][0][0]][0]

[1102, (126.97026536870001, 37.5832657654)]

In [501]:
bus[bus2[merge['ars_id'][13611]][0][0]][0][1][0],bus[bus2[merge['ars_id'][13611]][0][0]][0][1][1] ## 7022번 버스 노선 인덱스 0번 (1102)정류장의 좌표

(126.97026536870001, 37.5832657654)

In [542]:
## 버스정류장 좌표 근처 
ars = []
for i in range(len(bus[bus2[merge['ars_id'][13611]][0][0]])):
  ars.append(square(bus[bus2[merge['ars_id'][13611]][0][0]][i][1][0],bus[bus2[merge['ars_id'][13611]][0][0]][i][1][1],xxx,yyy))


In [543]:
print(ars)

[[[126.96936604870001, 37.5832657654], [126.97116468870001, 37.5832657654], [126.97026536870001, 37.581397915400004], [126.97026536870001, 37.5851336154]], [[126.9701443371, 37.585113193000005], [126.9719429771, 37.585113193000005], [126.9710436571, 37.58324534300001], [126.9710436571, 37.586981043]], [[126.9716141895, 37.5878519552], [126.9734128295, 37.5878519552], [126.9725135095, 37.585984105200005], [126.9725135095, 37.5897198052]], [[126.96622686049999, 37.5921866176], [126.96802550049999, 37.5921866176], [126.96712618049999, 37.5903187676], [126.96712618049999, 37.594054467599996]], [[126.9662913425, 37.5923353152], [126.9680899825, 37.5923353152], [126.9671906625, 37.59046746520001], [126.9671906625, 37.5942031652]], [[126.97156199979999, 37.5881285775], [126.97336063979999, 37.5881285775], [126.97246131979999, 37.586260727500004], [126.97246131979999, 37.5899964275]], [[126.9702553593, 37.584966342600005], [126.9720539993, 37.584966342600005], [126.9711546793, 37.5830984926000

In [614]:
## 좌표 내 100미터의 버스정류장 찾고
## 해당 정류장의 버스들의 노선에 있는 버스 정류장을 중심으로 하는 100m^2 정사각형 범위의 꼭지점 좌표구하기

def all_bus_line(x,y,df=merge):
  r500 = round500(x, y, df,True) ## 좌표 내 100미터 주변 버스정류장의 df 인덱스 리스트
  for stop in r500: 
    bus_id = df['ars_id'][stop] ## 인덱스에 해당하는 버스정류장 id
    bus_list=bus2[merge['ars_id'][bus_id]] ## 버스정류장 id에 해당하는 버스 번호 리스트
    for bus_nums in bus_list:
      lines=bus[bus_nums[0]] ## 버스 번호에 해당하는 노선 전체의 버스 정류장 id 리스트
      for line in lines: 
        result.append(square(line[1][0],line[1][1],xxx,yyy)) ## 버스정류장을 중심으로 하는 100m^2의 정사각형 범위의 좌표
        break
  return result



In [616]:
all_bus_line(metro['2호선'][0][0][1][0],metro['2호선'][0][0][1][1])[:5]

[[[127.0056918194, 37.5724174899],
  [127.0074904594, 37.5724174899],
  [127.0065911394, 37.5705496399],
  [127.0065911394, 37.574285339899994]],
 [[127.0648972607, 37.6143314819],
  [127.0666959007, 37.6143314819],
  [127.0657965807, 37.612463631900006],
  [127.0657965807, 37.6161993319]],
 [[127.0648972607, 37.6143314819],
  [127.0666959007, 37.6143314819],
  [127.0657965807, 37.612463631900006],
  [127.0657965807, 37.6161993319]],
 [[127.03491796379998, 37.611782602699996],
  [127.03671660379999, 37.611782602699996],
  [127.03581728379999, 37.6099147527],
  [127.03581728379999, 37.61365045269999]],
 [[127.0684104113, 37.651478387800005],
  [127.0702090513, 37.651478387800005],
  [127.0693097313, 37.64961053780001],
  [127.0693097313, 37.6533462378]]]